# Для SBER не корректно считается SELL в статистике #30 

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from app import create_app
from app.models import Instrument
from app.config import RunConfig
from bot.env.prod import TimeProdEnvHelper, LoggerHelper, TinkoffProxyClient, AccountingHelper
from bot import TradingBot

app = create_app()
with app.app_context():
    TICKER = 'SBER'
    INSTRUMENT_ID = 3
    
    log_name = TICKER
    
    instrument = Instrument.get_by_id(INSTRUMENT_ID)
    account_id = str(instrument.account)
    
    
    time_helper = TimeProdEnvHelper()
    logger_helper = LoggerHelper(__name__, log_name)
    client_helper = TinkoffProxyClient(TICKER, time_helper, logger_helper, account_id)
    accounting_helper = AccountingHelper(TICKER, client_helper)
    
    bot = TradingBot(
        config=RunConfig(
            ticker=TICKER,
    
            step_max_cnt=3,
            step_base_cnt=0,
            step_set_orders_cnt=2,
            step_size=1.2,
            step_lots=1,
    
            pretest_period=0,
            pretest_type=RunConfig.PRETEST_NONE,
    
            threshold_sell_steps=0,
            threshold_buy_steps=6,
    
            majority_trade=True,
    
            stop_up_p=.05,
            stop_down_p=.15,
    
            instrument_id=INSTRUMENT_ID,
        ),
        time_helper=time_helper,
        logger_helper=logger_helper,
        client_helper=client_helper,
        accounting_helper=accounting_helper,
    )

2024-08-28 22:48:36,021 - ERROR - Ошибка при запросе текущей цены для FIGI BBG004730N88
2024-08-28 22:48:36,022 - ERROR - Ошибка первичного запроса цены. Статистика будет неверной в конце работы
2024-08-28 22:48:37,218 - INFO - INIT 
     config - SBER+ 3/0/2 x l1 x 1.2¤ |s0 b6| |u0.05 d0.15| 
     instrument - SBER / BBG004730N88, Сбер Банк, rub, round_signs=2, inc=0.01, lot=10, short=True
     cur_used_cnt - 0
     last_price - 0
     depo - 0.0
     instrument_id - 3
     run_instance - <Run 82 i3 (SBER+ 3/0/2 x l1 x 1.2¤ |s0 b6| |u0.05 d0.15| ) at 2024-08-28>


In [7]:
buy_res = bot.trade_strategy.buy(1)
print(buy_res)

2024-08-28 18:18:40,762 - INFO - BUY MARKET executed, 1 x -260.69 | s1 (x1=1) | p -0.09 rub


PostOrderResponse(order_id='51395137207', execution_report_status=<OrderExecutionReportStatus.EXECUTION_REPORT_STATUS_FILL: 1>, lots_requested=1, lots_executed=1, initial_order_price=MoneyValue(currency='rub', units=2614, nano=800000000), executed_order_price=MoneyValue(currency='rub', units=260, nano=690000000), total_order_amount=MoneyValue(currency='rub', units=2606, nano=900000000), initial_commission=MoneyValue(currency='rub', units=0, nano=0), executed_commission=MoneyValue(currency='rub', units=0, nano=0), aci_value=MoneyValue(currency='', units=0, nano=0), figi='BBG004730N88', direction=<OrderDirection.ORDER_DIRECTION_BUY: 1>, initial_security_price=MoneyValue(currency='rub', units=261, nano=480000000), order_type=<OrderType.ORDER_TYPE_MARKET: 2>, message='', initial_order_price_pt=Quotation(units=0, nano=0), instrument_uid='e6123145-9665-43e0-8413-cd61b8aa9b13', order_request_id='2024-08-28 15:18:40.047686+00:00', response_metadata=ResponseMetadata(tracking_id='7fd691842f3a352

In [8]:
sell_res = bot.trade_strategy.sell(1)
print(sell_res)

2024-08-28 18:19:07,559 - INFO - SELL MARKET executed, 1 x 260.64 | s0 (x1=0) | p -0.05 rub


PostOrderResponse(order_id='RINX76QQ6JH1TYW0795V', execution_report_status=<OrderExecutionReportStatus.EXECUTION_REPORT_STATUS_FILL: 1>, lots_requested=1, lots_executed=1, initial_order_price=MoneyValue(currency='rub', units=2598, nano=400000000), executed_order_price=MoneyValue(currency='rub', units=260, nano=640000000), total_order_amount=MoneyValue(currency='rub', units=2606, nano=400000000), initial_commission=MoneyValue(currency='rub', units=0, nano=0), executed_commission=MoneyValue(currency='rub', units=0, nano=0), aci_value=MoneyValue(currency='', units=0, nano=0), figi='BBG004730N88', direction=<OrderDirection.ORDER_DIRECTION_SELL: 2>, initial_security_price=MoneyValue(currency='rub', units=259, nano=840000000), order_type=<OrderType.ORDER_TYPE_MARKET: 2>, message='', initial_order_price_pt=Quotation(units=0, nano=0), instrument_uid='e6123145-9665-43e0-8413-cd61b8aa9b13', order_request_id='2024-08-28 15:19:06.770864+00:00', response_metadata=ResponseMetadata(tracking_id='9f0f9

In [9]:
buy_l_res = bot.trade_strategy.buy_limit(261, 1)
print(buy_l_res)

2024-08-28 18:19:54,048 - INFO - Buy order set, 1 x -2610.0 | s0 (x1=0) | p -0.05 rub


PostOrderResponse(order_id='51395228575', execution_report_status=<OrderExecutionReportStatus.EXECUTION_REPORT_STATUS_NEW: 4>, lots_requested=1, lots_executed=0, initial_order_price=MoneyValue(currency='rub', units=2610, nano=0), executed_order_price=MoneyValue(currency='', units=0, nano=0), total_order_amount=MoneyValue(currency='rub', units=2610, nano=0), initial_commission=MoneyValue(currency='', units=0, nano=0), executed_commission=MoneyValue(currency='', units=0, nano=0), aci_value=MoneyValue(currency='', units=0, nano=0), figi='BBG004730N88', direction=<OrderDirection.ORDER_DIRECTION_BUY: 1>, initial_security_price=MoneyValue(currency='rub', units=261, nano=0), order_type=<OrderType.ORDER_TYPE_LIMIT: 1>, message='', initial_order_price_pt=Quotation(units=0, nano=0), instrument_uid='e6123145-9665-43e0-8413-cd61b8aa9b13', order_request_id='2024-08-28 15:19:53.617848+00:00', response_metadata=ResponseMetadata(tracking_id='34ace867293e0c6282c7319dae7f62c8', server_time=datetime.date

In [10]:
bot.trade_strategy.update_orders_status()

2024-08-28 18:20:24,144 - INFO - BUY order executed, 1 x 2608.2 | s1 (x1=1) | p -2348.95 rub
2024-08-28 18:20:24,691 - ERROR - Не выставлена заявка: order_type: limit, direction: sell, lots: 1, price: 2609.4, Error: (<StatusCode.INVALID_ARGUMENT: (3, 'invalid argument')>, '30099', Metadata(tracking_id='01660e5ff0f06ff1ebaa948727477e3a', ratelimit_limit='300, 300;w=60', ratelimit_remaining=299, ratelimit_reset=36, message='The price is outside the limits for this instrument'))
2024-08-28 18:20:24,692 - ERROR - RETRY order. 1, 2, 2609.3999999999996, 1, sleep 5, retry num=1
2024-08-28 18:20:30,032 - ERROR - Не выставлена заявка: order_type: limit, direction: sell, lots: 1, price: 2609.4, Error: (<StatusCode.INVALID_ARGUMENT: (3, 'invalid argument')>, '30099', Metadata(tracking_id='e4c9ce21b3dcd330fb6110d902fa55e8', ratelimit_limit='300, 300;w=60', ratelimit_remaining=298, ratelimit_reset=31, message='The price is outside the limits for this instrument'))
2024-08-28 18:20:30,187 - INFO - O

In [14]:
buy_l_executed, buy_l_state = bot.client.order_is_executed(buy_l_res)
print(buy_l_executed)
print(buy_l_state)

True
OrderState(order_id='51395228575', execution_report_status=<OrderExecutionReportStatus.EXECUTION_REPORT_STATUS_FILL: 1>, lots_requested=1, lots_executed=1, initial_order_price=MoneyValue(currency='rub', units=2610, nano=0), executed_order_price=MoneyValue(currency='rub', units=2608, nano=200000000), total_order_amount=MoneyValue(currency='rub', units=2608, nano=200000000), average_position_price=MoneyValue(currency='rub', units=260, nano=820000000), initial_commission=MoneyValue(currency='rub', units=1, nano=310000000), executed_commission=MoneyValue(currency='rub', units=1, nano=300000000), figi='BBG004730N88', direction=<OrderDirection.ORDER_DIRECTION_BUY: 1>, initial_security_price=MoneyValue(currency='rub', units=261, nano=0), stages=[OrderStage(price=MoneyValue(currency='rub', units=260, nano=820000000), quantity=1, trade_id='11113121741', execution_time=datetime.datetime(2024, 8, 28, 15, 19, 54, 49836, tzinfo=datetime.timezone.utc))], service_commission=MoneyValue(currency='

In [13]:
print(bot.trade_strategy.get_current_profit(bot.trade_strategy.get_current_price()))

-2348.51
